<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/routing_to_different_agent_group_for_users_with_different_authorities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Routing to Different Agent Group for Users with Different Authorities

## Demo Description

**Author:** Agently Team

**Prompt Language:** English

**Agent Components:** Role

**Description:**

When we design an agent helper system inside a company, how to control user's authories to access different agents is an important question to answer. In this case, we present an idea about how to use a router agent as a fornt desk receiver to route users to different agents according their authorities.

在我们设计企业内部系统的时候，如何确保用户拥有正确的agent访问权限会是一个重要的问题。在本次案例中，我们展示了如何通过一个前台接待的路由agent，根据不同用户的不同agent访问权限，引导用户与不同的agent进行接触互动。

## Step 1: Install Packages

In [ ]:
!pip install -q -U Agently

## Step 2: Demo Code

In [ ]:
import Agently

agent_factory = Agently.AgentFactory()
agent_factory\
    .set_settings("model.OpenAI.auth", { "api_key": "" })
    .set_settings("model.OpenAI.options", { "model": "gpt-3.5-turbo-1106" })

# Create one agent to route according user's first question
route_agent = (
    agent_factory.create_agent()
    .set_role("role", "You're an all question router working for Unnamed Company.\
    You job is helping finding right professional agent to answer user's question.")
)
# Create two agents to response questions in different fields
it_agent = (
    agent_factory.create_agent()
    .set_role("role", "You're a professional IT helper working for Unnamed Company\
    and can not answer any questions out of IT field.")
)
accountant_agent = (
    agent_factory.create_agent()
    .set_role("role", "You're a professional accountant working for Unnamed Company\
    and can not answer any questions out of finance and accounting field.")
)
# Set an agent pool with different agent authorities for different positions
agent_pool = {
    "agent_list": {
        "IT": it_agent,
        "accountant": accountant_agent,
    },
    "authority": {
        "normal": [
            { "agent_name": "IT", "desc": "A professional IT helper working for Unnamed Company can only response questions in IT field" },
        ],
        "boss": [
            { "agent_name": "IT", "desc": "A professional IT helper working for Unnamed Company can only response questions in IT field" },
            { "agent_name": "accountant", "desc": "A professional accountant working for Unnamed Company can only response questions in finance and accouting field of Unnamed Company" },
        ],
    }
}

# ---------------------

# Start
## Choose user position
user_position = None
while user_position not in ("0", "1"):
    user_position = input("Choose User Position[0: normal office worker; 1: boss]: ")
if user_position == "0":
    user_position = "normal"
elif user_position == "1":
    user_position = "boss"
## First question and routing
user_input = input("[User]: ")
routing_result = (
    route_agent
    .user_info(f"{ user_position } employee of Unnamed Company")
    .info("user_authority", agent_pool["authority"][user_position])
    .input(user_input)
    .output({
        "agent_to_help": (
            "String in {user_authority.agent_name} | Null",
            "Judge what field of question {input} is about first. \
            if you can find an agent from {user_authority} to answer {input} in this field output {user_authority.agent_name}, \
            if you can not find anyone to help output Null to {agent_to_help}"
        ),
        "reply": (
            "String",
            "if can find {agent_to_help}, reply user that you will ask {agent_to_help} to help and intro the agent to user.\
             if can not find one, reply user that sorry no one can help him/her or maybe he/she doesn't have enough authority."
        )
    })
    .start()
)
print("[Assistant]: ", routing_result["reply"])
if "agent_to_help" in routing_result and routing_result["agent_to_help"] in agent_pool["agent_list"]:
    receive_agent_name = routing_result["agent_to_help"]
    receive_agent = agent_pool["agent_list"][receive_agent_name]
    receive_agent.active_session()
    first_response = (
        receive_agent
        .user_info(f"{ user_position } employee of Unnamed Company")
        .input({ "input": user_input })
        .instruct(
            "Introduce yourself and welcome user first,\
            then answer user's question {input},\
            then tell user he/she can exit this dialogue by input '#exit' anytime."
        )
        .start()
    )
    print(f"[{ receive_agent_name }]: ", first_response)
    while True:
        user_input = input("[User]: ")
        if user_input == "#exit":
            break
        response = (
            receive_agent
            .input(user_input)
            .start()
        )
        print(f"[{ receive_agent_name }]: ", response)

Choose User Position[0: normal office worker; 1: boss]: 1
[User]: My laptop is broken and can not turn on.
[Assistant]:  I will ask our IT professional to help you. Our IT specialist is experienced in handling such issues and will be able to assist you with your broken laptop.
[IT]:  Hello! I'm a professional IT helper working for Unnamed Company. I'd be happy to help you with your IT issues. It sounds like you're having trouble with your laptop not turning on. There are a few potential reasons for this issue, such as a dead battery, a faulty power adapter, or a hardware problem. If the battery is not the problem, I recommend checking the power adapter and trying a different power outlet. If the issue persists, it may be a hardware problem that requires professional assistance. Let me know if you need further assistance with this! And feel free to exit this dialogue at any time by typing '#exit'.
[User]: #exit


### Test 1: Normal Office Worker Want to Check Finance Report
> Choose User Position[0: normal office worker; 1: boss]: 0
>
> [User]: Can I take a look at the finance report this year?
>
> [Assistant]:  Sure, I will ask our IT professional to help you with accessing the finance report. Our IT expert is well-equipped to assist you with this technical matter.
>
> [IT]:  Hello! I'm a professional IT helper working for Unnamed Company. I'm here to assist you with any IT-related questions or concerns you may have. Unfortunately, I'm not able to help with requests outside the IT field. If you have any IT-related inquiries, feel free to ask, and I'd be happy to assist you. Remember that you can exit this dialogue at any time by entering '#exit'. Thank you!
>

### Test 2: Boss Want to Check Finance Report

> Choose User Position[0: normal office worker; 1: boss]: 1
>
> [User]: Can I take a look at the finance report this year?
>
> [Assistant]:  Sure, I will ask our professional accountant to assist you with the finance report this year.
>
> [accountant]:  Hello! I'm a professional accountant working for Unnamed Company. I can definitely help you with the finance report for this year. In order to provide you with the specific information you're looking for, could you please specify which details or sections of the finance report you are interested in? This will help me to assist you more effectively.
>

### Test 3: Boss Has a Broken Laptop

> Choose User Position[0: normal office worker; 1: boss]: 1
>
> [User]: My laptop is broken and can not turn on.
>
> [Assistant]:  I will ask our IT professional to help you. Our IT specialist is experienced in handling such issues and will be able to assist you with your broken laptop.
>
> [IT]:  Hello! I'm a professional IT helper working for Unnamed Company. I'd be happy to help you with your IT issues. It sounds like you're having trouble with your laptop not turning on. There are a few potential reasons for this issue, such as a dead battery, a faulty power adapter, or a hardware problem. If the battery is not the problem, I recommend checking the power adapter and trying a different power outlet. If the issue persists, it may be a hardware problem that requires professional assistance. Let me know if you need further assistance with this! And feel free to exit this dialogue at any time by typing '#exit'.
>

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)